# Extractive Summarizer with BERT

In [1]:
!pip install transformers kss sentencepiece

     |████████████████████████████████| 1.8MB 15.2MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 1.2MB 68.8MB/s 
     |████████████████████████████████| 890kB 63.6MB/s 
     |████████████████████████████████| 2.9MB 66.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e625e689347e33b70dd708f9a69cd1cc4d3e4dc80fea08b583d094799c0670fa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import math
from sklearn.cluster import KMeans
from transformers import BertModel, BertTokenizer, AutoTokenizer, AlbertPreTrainedModel
import torch
import kss

In [3]:
def euclideanDistance(sample1,sample2):
    
    sum = 0
    for i in range(sample1.shape[0]):
        number = sample1[i]-sample2[i]
        sum = sum + math.pow(number,2)
            
    return math.pow(sum,0.5)

def kMeans(features, input,k):

    features = features.reshape((features.shape[0],features.shape[1]*features.shape[2]))

    model = KMeans(n_clusters=k)
    labels = model.fit_predict(features)
    cluster_centers = model.cluster_centers_

    distance_list = []

    for center in list(cluster_centers):
        distances = {}
        for i, sentence in enumerate(features):
            distances[str(i)] = euclideanDistance(center,sentence)

        import operator 
        sorted_distances = sorted(distances.items(), key=operator.itemgetter(1))
        distance_list.append(sorted_distances)

    idx = [int(value[0][0]) for value in distance_list]
    idx = sorted(idx)
    
    return idx

# Test

In [4]:
from transformers import AutoTokenizer, AutoModel

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# My albert model
# model = AutoModel.from_pretrained('halo/model')
# tokenizer = AutoTokenizer.from_pretrained('halo/model')
def BERTFeatureExtraction(input, max_token_size):
    s = []
    temp = 0
    for sent in kss.split_sentences(input):
        s.append(sent)
        # print(sent)
    input = s
    input = input[:-1]
    print("Number of sentences in the input text:", len(input))

    features = []
    
    for sentence in (input):
        tokens = tokenizer.tokenize(sentence)
        special_tokens = ['[CLS]'] + tokens + ['[SEP]']

        if len(special_tokens)>max_token_size:
            print("Warning max_token_size is small. Change the parameter! Maybe be there is no dot in the input text.")
            
            
        padded_tokens = special_tokens + ['[PAD]' for i in range(max_token_size-len(special_tokens))]
        token_ids = tokenizer.convert_tokens_to_ids(padded_tokens)
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in padded_tokens]
        
        token_ids = torch.tensor(token_ids).unsqueeze(0)
        attn_mask = torch.tensor(attn_mask).unsqueeze(0)

        # Old version 
        # hidden_repr, cls_head = model(token_ids, attention_mask = attn_mask)

        features.append(model(token_ids, attention_mask = attn_mask)[0]) #batch dimension reduction (1, token_size, wv) => (token_size, wv)
        temp = token_ids
        temp2 = attn_mask
    
    import numpy as np
    features = [value.detach().numpy() for value in features]
    features = [value.reshape((max_token_size,768)) for value in features]
    features = np.array(features).reshape((len(features),max_token_size,768))
    
    
    return features, input, temp, temp2

# Model Load

In [5]:
#MAIN************************************************************************

go = '''
칼리스토(영어: Callisto/kəˈlɪstoʊ/, 그리스어: Καλλιστώ) 또는 목성 IV는 목성의 위성으로, 1610년에 갈릴레오 갈릴레이가 발견했다. 또한 칼리스토는 태양계에서 3번째로 큰 위성이고, 목성의 위성 중에서는 가니메데 다음으로 크며, 행성 분화율은 태양계에서 제일 낮다. 그리고 칼리스토의 지름은 4,821km이며, 수성의 99%에 달하는 크기를 가졌지만, 칼리스토의 질량은 3분의 1밖에 되지 않는다. 또한 궤도 반경은 1,880,000km이고, 갈릴레이 위성 중에서는 목성으로부터 제일 멀리 있다.[2] 칼리스토는 내부 갈릴레이 위성들(이오, 유로파, 가니메데)과 궤도 공명을 일으키지 않고, 따라서 조석 가열의 정도가 약하다.[8] 칼리스토는 목성에 조석 고정되어 있기 때문에, 항상 같은 면만 목성을 바라보게 된다. 칼리스토는 다른 목성의 위성들에 비해 자기권이 약한데, 이는 목성의 방사선대에서 멀리 떨어져 있기 때문이다.[9][10]

칼리스토는 거의 같은 비율의 암석과 얼음으로 구성되어 있으며, 밀도는 1.83 g/cm3으로 목성의 주요 위성들 중 밀도가 가장 낮고 표면 중력 또한 가장 약하다. 분광기를 사용해 표면을 분석했을 때 발견된 물질들로는 얼음,[11] 이산화 탄소, 규산염, 유기 화합물들이 있었다. 갈릴레오 탐사선은 칼리스토가 작은 규산염 핵이 있을 수도 있다는 사실과 칼리스토가 깊이 100 km가 넘는 지하수가 존재할 가능성을 밝혀냈다.[11][12][13]

칼리스토의 표면은 태양계에서 가장 오래 되었고 충돌구가 가장 많이 존재한다.[14] 또한 판이나 화산과 같은 어떠한 지질학적 활동도 보여주지 않고 이러한 활동이 일어난 흔적도 없으며, 표면은 주로 충돌을 통해 변화해 왔다고 추측된다.[15] 표면의 주요 구조물로는 다환 충돌구(multi-ring structures), 다양한 형태의 충돌구들, 사슬형 충돌구(카테나), 그리고 이러한 충돌이 빚어낸 산등성이, 단애, 퇴적지형이 있다.[15] 작은 규모에서 보면, 서리로 덮여 밝게 빛나는 돌출된 지형과, 이 주변을 덮은 어둡고 부드러운 퇴적층으로 구분된다.[4] 이는 승화가 촉진한 풍화 작용이 작은 충돌구를 없애고, 그 결과 많은 수의 돌출 지형을 남긴 것으로 보인다.[16] 이 지형들의 정확한 연대는 아직 알려지지 않았다.

또한 칼리스토는 이산화탄소와 (아마도) 산소로 이루어진 매우 옅은 대기권으로 둘러싸여 있고,[6][7] 상당히 강한 전리층도 존재한다.[17] 칼리스토는 목성이 형성될 때 주변에 형성되었던 가스와 먼지 원반에서 느린 강착을 통해서 형성되었다고 추측되고 있다.[18] 칼리스토의 점진적인 강착 속도와 조석 가열량의 부족으로 인해 칼리스토는 빠른 행성 분화가 일어날 만한 열이 부족한 상태였다. 형성 직후 시작된 칼리스토 내부에서의 대류는 불완한 행성 분화를 일으켰고, 이에 따라 작고 암석질인 핵과 깊이 100~150 km의 지하 바다가 형성되었다고 추측된다.[19]

만일 칼리스토에도 바다가 존재한다면, 다른 천체들처럼 외계 생명체가 존재할 가능성이 있지만, 환경은 근처의 유로파보다 좋지 않으리라고 추측된다.[20] 칼리스토를 연구해 온 탐사선으로는 파이어니어 10호, 파이어니어 11호, 갈릴레오, 카시니-하위헌스 등이 있다. 또한 칼리스토는 방사능이 매우 낮아, 미래 유인 목성계 탐사 시 기지를 세울 장소로 고려되고 있다.[21]
'''
print(len(go))
max_token_size = 512
k=4

features, go, temp, temp2 = BERTFeatureExtraction(go, max_token_size) #input is string, output is (batch_size=num_sen, token_size, hidden_dim)
idx = kMeans(features, go, k)


summary = [go[id]+"." for id in(idx)]
for sentence in summary:
    print(sentence)

1673
Number of sentences in the input text: 23
칼리스토는 다른 목성의 위성들에 비해 자기권이 약한데, 이는 목성의 방사선대에서 멀리 떨어져 있기 때문이다..
[11][12][13]

칼리스토의 표면은 태양계에서 가장 오래 되었고 충돌구가 가장 많이 존재한다..
[14] 또한 판이나 화산과 같은 어떠한 지질학적 활동도 보여주지 않고 이러한 활동이 일어난 흔적도 없으며, 표면은 주로 충돌을 통해 변화해 왔다고 추측된다..
[15] 작은 규모에서 보면, 서리로 덮여 밝게 빛나는 돌출된 지형과, 이 주변을 덮은 어둡고 부드러운 퇴적층으로 구분된다..


In [6]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         